In [1]:
import gmql as gl

In [2]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result"

parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path, meta_load=False)
    
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (data.MetaField("cell") == 'K562') & (data.MetaField("antibody") == 'H3K4me3')
filtered_data = data.meta_select(predicate=condition) # test

In [3]:
# project on region fields
new_reg_field = {
    "new_attribute" : (filtered_data.RegField("pValue") + filtered_data.RegField("signalValue"))/2
}

In [4]:
proj_data = filtered_data.reg_project(field_list=['score', 'signalValue', 'pValue'], new_field_dict=new_reg_field)

In [5]:
# cover operation

new_fields = {
    "min_pValue" : gl.MIN("pValue")
}
cover_data = proj_data.normal_cover(minAcc=1, maxAcc=2, groupBy=['treatment_tag'], new_reg_fields=new_fields)

In [ ]:
# materialization
cover_data = cover_data.materialize(output_path=output_path)